# Arboles de Decisión


In [2]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config

from sklearn.tree import DecisionTreeClassifier
from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Creando dataframe..."


builder = DataFrameBuilder()
df = builder.build()

print "Listo"

Creando dataframe...
Listo


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [3]:
help(DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree.tree:

class DecisionTreeClassifier(BaseDecisionTree, sklearn.base.ClassifierMixin)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : string, optional (default="best")
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_features : int, float, string or None, optional (default=None)
 |      The number of features to consider when looking for the best split:
 |        - If int, then consider `max_features` features at each split.
 |        - If float, then `max_features` is a percentage and
 |         


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [4]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import cross_val_score

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes

scoring_methods = ['precision', 'accuracy', 'f1', 'recall', 'roc_auc']

for scoring in scoring_methods:
    clf = DecisionTreeClassifier()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, cv=10, n_jobs=4, n_iter=100)
    
    search.fit(X, y)
    
    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}\n\n".format(search.best_score_)

    """
    Esto puede no tener mucho sentido. Estamos corriendo nuevamente cross_val_score... 
    """
    for other_scorer in scoring_methods:
        print "Valor para {} = {}".format(
            other_scorer,
            cross_val_score(search.best_estimator_, X, y, scoring=other_scorer, cv=10).mean()
        )
        
    

    


Scoring precision


/home/marat/.pyenv/versions/2.7.11/envs/aprendizaje/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Mejor combinación: {'min_samples_split': 12, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 42, 'max_features': 0.90000000000000013}
Mejor valor: 0.965353161617


Valor para precision = 0.965135533264
Valor para accuracy = 0.963033561528
Valor para f1 = 0.963532576743
Valor para recall = 0.96117819672
Valor para roc_auc = 0.975875634472

Scoring accuracy
Mejor combinación: {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 42, 'max_features': 0.50000000000000011}
Mejor valor: 0.963948391876


Valor para precision = 0.963694098701
Valor para accuracy = 0.964169435641
Valor para f1 = 0.96330809378
Valor para recall = 0.964462665679
Valor para roc_auc = 0.964080489497

Scoring f1


/home/marat/.pyenv/versions/2.7.11/envs/aprendizaje/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Mejor combinación: {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 42, 'max_features': 0.90000000000000013}
Mejor valor: 0.965241354235


Valor para precision = 0.964938914345
Valor para accuracy = 0.964811476497
Valor para f1 = 0.96381447389
Valor para recall = 0.964783842375
Valor para roc_auc = 0.964497050216

Scoring recall
Mejor combinación: {'min_samples_split': 12, 'splitter': 'random', 'criterion': 'entropy', 'max_depth': 34, 'max_features': 0.10000000000000001}
Mejor valor: 0.963748360624


Valor para precision = 0.855210716903
Valor para accuracy = 0.796525256128
Valor para f1 = 0.788897953243
Valor para recall = 0.879790583866
Valor para roc_auc = 0.907230702747

Scoring roc_auc
Mejor combinación: {'min_samples_split': 72, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 26, 'max_features': 0.90000000000000013}
Mejor valor: 0.986701092515


Valor para precision = 0.95716742471
Valor para accuracy = 0.959094860057
Valor para f1 = 0.95